# Deploying models to Azure ACI using Azure ML

Method:
1. Train scikit-learn model
2. (Optional) Track experiments with Azure ML workspace
3. Register model to Azure ML workspace
4. Deploy as web service to Azure ACI

## 1. Train scikit-learn model

In [ ]:
import numpy as np
import sklearn
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
import joblib

X_train, y_train, X_test, y_test = [np.load("../data/diabetes.npz")[x] for x in ("X_train", "y_train", "X_test", "y_test")]

alpha=0.1

model = Ridge(alpha=alpha).fit(X_train, y_train)

joblib.dump(model, "diabetes_model.pkl")

## 1.1 (Alternative) Download model from remote MLFlow

Note to be made here: Azure MLFlow deployment client only accepts models tracked in a native Azure MLFlow (see [02 - Deploy MLFlow endpoint](../02%20-%20Deploy%20MLFlow%20endpoint/02%20-%20demo.ipynb))). For models tracked elsewhere (e.g. MLFlow on AWS), we need to download the model and deploy this way instead.

In [ ]:
import mlflow
import joblib, os
#os.environ["AWS_ACCESS_KEY_ID"] = "" #if needed, set AWS credentials in local environment
#os.environ["AWS_SECRET_ACCESS_KEY"] = ""
mlflow.set_tracking_uri("http://andre-mlflo-a3oi3g4g7sh1-6b98eef0c99b7004.elb.eu-central-1.amazonaws.com")
model = mlflow.sklearn.load_model(f"models:/diabetes/1")
joblib.dump(model, "diabetes_model.pkl")

# 2. (Optional) Track experiments with Azure ML workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
from azureml.core import Experiment

experiment = Experiment(workspace=ws, name="train-diabetes-regression")
run = experiment.start_logging()

run.log('alpha', alpha)
run.log('R2', r2_score(y_test, model.predict(X_test)))
run.upload_file(name="diabetes_model.pkl", path_or_stream="./diabetes_model.pkl")

run.complete()

# 3. Register model to Azure ML Workspace

In [ ]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
import sklearn

model = Model.register(workspace=ws,
                       model_name="diabetes_model",
                       model_path="./diabetes_model.pkl",
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version=sklearn.__version__, ## make sure sklearn version is same as sklearn in which model was trained!
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='Ridge regression to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

If the experiment is being tracked in Azure ML workspace (i.e. following Section 2), then the model can be directly registered from the run:

In [ ]:
run.register_model(model_path="./diabetes_model.pkl", model_name='diabetes_model')

# 4. Deploy as web service to Azure ACI

Below is a "no-code" approach - Azure ML will detect the environment and appropriate inference function (in this case, `sklearn.estimator.predict`)

In [ ]:
service = Model.deploy(ws, 'my-sklearn-service1', [model], overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

## 4.1 (Alternative) Deploy with specific inference function and environment

In [ ]:
%%writefile diabetes_model_inference.py
import json, joblib, os
import numpy as np

def init():
    global model
    model = joblib.load(os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl'))

def run(raw_data):
    predictions = model.predict(np.array(json.loads(raw_data)['data'])).tolist()
    return json.dumps(predictions)

In [ ]:
from azureml.core.model import InferenceConfig, Environment

service_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
service_env.inferencing_stack_version="latest"
inference_config=InferenceConfig(entry_script="diabetes_model_inference.py",
                                environment=service_env)

service = Model.deploy(ws, 'my-sklearn-service2', [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

# Test endpoint and delete

In [ ]:
import requests
import json

input_payload = json.dumps({
    'data': X_test[0:2].tolist(),
})

requests.post(endpoint, input_payload, headers={'Content-Type':'application/json'}).json()

Alternatively, if the service is still loaded in memory:

In [ ]:
service.run(input_payload)

Make sure to delete endpoint after use:

In [ ]:
service.delete()

# Manage

You can monitor and manage the endpoint under "Endpoints" in the Azure ML Studio UI.

# References

https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb

https://github.com/MicrosoftLearning/mslearn-dp100/blob/main/09%20-%20Create%20a%20Real-time%20Inferencing%20Service.ipynb